In [41]:
DATA_LINK = "https://docs.google.com/spreadsheets/d/1LYYyjZtW7cLXMN5mTh3-K1pBRn6ncA_T"
APPEND_TO_LINK = "/gviz/tq?tqx=out:html&tq&gid=1"
TABLE_LINK = DATA_LINK + APPEND_TO_LINK

In [42]:
import pandas as pd

In [43]:
df = pd.read_html(TABLE_LINK, skiprows=2, header=0)[0]

In [44]:
display(df.head())

,Ms ID #,Manuscript Title,Article Type,Corresponding Author Name,First Author Name,Corresponding Author Email,No. of supplementary files,Section,Unnamed: 8,Corresponding Author mailing address,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,paper_05,Gravity influence in one-dimensional blood flo...,research,Oleg Shramko,Oleg Shramko,oashramko@itmo.ru,https://github.com/ITMO-MMRM-lab/Complex_blood...,Healthcare,1.0,"ITMO University, Kronverksky Pr. 49, bldg. A, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,paper_06,Light-weight ensembling of deep neural models ...,research,Anna Litvintseva,Ilya Revin,avlitvintseva@itmo.ru,https://github.com/ITMO-NSS-team/LightObjRecEn...,Deep Learning and Data-Driven Modelling,36.0,"ITMO university, Kronverksky Pr. 49, bldg. A, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,paper_10,Comparative Analysis of SVM Classifiers in Cri...,research,NaN,NaN,NaN,NaN,Data Analysis and NLP,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paper_12,"Hybrid Algorithm for Multi-Contractor, Multi-R...",research,Aleksandr Voskresenskii,Aleksandr Voskresenskii,asvoskresenskii@gmail.com,NaN,Optimisation and Scheduling,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,paper_14,Framework architecture of a secure big data lake,research,NaN,NaN,NaN,https://github.com/IcyAltair/Pet-projects/tree...,Computing infrastructure,18.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
def leave_only_number(text):
    return text.split("_")[1]

import re
def parse_links(text):
    links = re.findall(r'(https?://\S+)', str(text).rstrip("."))
    return links

def link_to_badge(links):
    if links:
        links[0] = f"[![Code](https://img.shields.io/badge/Code-159957.svg)]({links[0]})"
        try: 
            links[1] = f"[![Data](https://img.shields.io/badge/Data-20BEFF.svg)]({links[1]})"
        except IndexError:
            pass
        return '  '.join(links)

def generate_section_list(f, groups):
    f.writelines('\n## Papers\n\n')
    f.writelines('<details open>\n')
    f.write('<summary>List of sections<a id="sections"></a></summary>\n\n')
    [f.write(title_to_link(name)) for (name, _) in groups]
    f.write('</details>\n\n')

def title_to_link(title):
    return f"- [{title}](#{title.lower().replace(' ', '-')})\n\n"

def generate_sections(f, groups):
    for (name, g_df) in groups:
        f.write(f"## {name}\n\n")
        f.write(generate_section_badges(g_df))
        f.write(g_df[["ID","Title","Links"]].fillna('').to_markdown(index=False))
        f.write("\n\n")
        f.write(back_to_sections())

import numpy as np
def generate_section_badges(df):
    return f"![Section Papers](https://img.shields.io/badge/Section%20Papers-{df['ID'].count()}-42BA16) ![Papers with Open Code](https://img.shields.io/badge/Papers%20with%20Open%20Code-{df['Links'].replace('', np.nan).count()}-1D7FBF)\n"
    

def back_to_sections():
    return f"[Back to top](#papers)\n\n"

In [46]:
# Generate List of Sections

df = df.rename(columns={"Ms ID #": "ID", "Manuscript Title":"Title &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;", "No. of supplementary files":"Links &nbsp;"})
df["ID"] = df["ID"].apply(lambda cell: leave_only_number(cell))
df["Links"] = df["Links"].apply(lambda cell: link_to_badge(parse_links(cell)))

groups = df.groupby("Section")

with open('generated.md', 'w') as f:
    generate_section_list(f,groups)
    generate_sections(f,groups)

KeyError: 'Links'

In [ ]:
# Merge base.md and generated.md into a single readme.md file

filenames = ["base.md", "generated.md"]
with open('../README.md', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)